In [ ]:
import pandas as pd 
import plotly.graph_objects as go
#import webcolors
import os

data_folder = r"/workspaces/Projects/Cashew_Board/data"
file_on_folder = os.listdir(data_folder)[0]

def android_to_hex(color_code):
    hex_code = "#{:06x}".format(color_code & 0xffffff)
    return hex_code

df = pd.read_csv(fr"{data_folder}/{file_on_folder}", engine = "pyarrow", keep_default_na = False)
df["color"] = df["color"].apply(android_to_hex)

,account,amount,currency,title,note,date,income,type,category name,subcategory name,color,icon,emoji,budget,objective
0,Main,-12.0,USD,,,2024-11-21 12:48:18,False,null,Taxi,,#ffee58,taxi(1).png,,,
1,Main,-8.0,USD,,,2024-11-21 12:48:14,False,null,Taxi,,#ffee58,taxi(1).png,,,
2,Main,-8.0,USD,,,2024-11-21 07:12:03,False,null,Taxi,,#ffee58,taxi(1).png,,,
3,Main,-8.0,USD,,,2024-11-21 07:11:59,False,null,Taxi,,#ffee58,taxi(1).png,,,
4,Main,-20.0,USD,,Proton Duo wi,2024-11-20 18:31:51,False,null,Subscription,,#ff7043,subscription.png,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,Main,-1.0,USD,,,2024-01-03 06:46:28,False,null,Groceries,,#4caf50,groceries.png,,,
1284,Main,10.0,USD,,,2024-01-02 16:38:20,True,null,Tip,,#66bb6a,dollar-coin.png,,,
1285,Main,-16.0,USD,,,2024-01-02 16:37:50,False,null,Taxi,,#ffee58,taxi(1).png,,,
1286,Main,-14.0,USD,,,2024-01-02 16:37:26,False,null,Taxi,,#ffee58,taxi(1).png,,,


In [ ]:
def plot_pie_categories(df):
    
    df = df[df["income"] == False] # Keeps only expenses
    df['amount'] = df['amount'].abs() # Transform negative values to positive
    df["category name"] = df.apply(lambda row: row["category name"] if row["subcategory name"] == "" else row["subcategory name"], axis = 1) # Keep subcategory if not empty
    df = df[["category name", "amount", "color"]].groupby(["category name"]).agg({"amount": "sum", "color": "first"}).reset_index() # Group by category name and sum amount
    
    fig = go.Figure(layout = go.Layout(height = 500, width = 500))
    fig.add_trace(go.Pie(labels = df["category name"],
                         values = df["amount"],
                         hoverinfo = "label+value+percent",
                         textinfo = "label+value",
                         textposition = "inside",
                         showlegend = False,
                         marker = dict(colors = df["color"],)))
    
    return fig

plot_pie_categories(df)


/tmp/ipykernel_34235/1939136839.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_34235/1939136839.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

